In [1]:
import pandas as pd

In [2]:
df= pd.read_csv(r'C:\Users\52296881\Downloads\churndata.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
geography = pd.get_dummies(df['Geography'],drop_first=True)
gender =pd.get_dummies(df['Gender'],drop_first=True)

In [5]:
df = pd.concat([df,geography,gender],axis= 1)

In [6]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Germany,Spain,Male
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [7]:
df.drop(labels= ['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [8]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [9]:
df.drop(labels= ['Geography','Gender'],axis=1,inplace=True)
y = df['Exited']
x = df.drop('Exited',axis =1)

In [10]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LeakyReLU,PReLU,ELU,Dropout
import joblib

In [17]:
ss = StandardScaler()
ss.fit(x)
joblib.dump(ss, 'StandardScalar')
x=ss.transform(x)

In [18]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [19]:
classifier = Sequential()

In [20]:
classifier.add(Dense(units=8,kernel_initializer = 'he_uniform',activation = 'relu',input_dim = 11))
classifier.add(Dense(units=6,kernel_initializer = 'he_uniform',activation = 'relu'))
classifier.add(Dense(units=4,kernel_initializer = 'he_uniform',activation = 'relu'))
classifier.add(Dense(units=1,activation = 'sigmoid'))
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [23]:
classifier.fit(X_train,Y_train,validation_split=0.1,batch_size = 10,epochs = 10)

Epoch 1/10
720/720 [==============================] - 1s 2ms/step - loss: 0.3448 - accuracy: 0.8636 - val_loss: 0.3784 - val_accuracy: 0.8375
Epoch 2/10
720/720 [==============================] - 4s 6ms/step - loss: 0.3429 - accuracy: 0.8642 - val_loss: 0.3758 - val_accuracy: 0.8388
Epoch 3/10
720/720 [==============================] - 3s 4ms/step - loss: 0.3427 - accuracy: 0.8633 - val_loss: 0.3782 - val_accuracy: 0.8413
Epoch 4/10
720/720 [==============================] - 2s 3ms/step - loss: 0.3406 - accuracy: 0.8637 - val_loss: 0.3804 - val_accuracy: 0.8413
Epoch 5/10
720/720 [==============================] - 1s 2ms/step - loss: 0.3405 - accuracy: 0.8647 - val_loss: 0.3781 - val_accuracy: 0.8425
Epoch 6/10
720/720 [==============================] - 4s 6ms/step - loss: 0.3394 - accuracy: 0.8640 - val_loss: 0.3752 - val_accuracy: 0.8438
Epoch 7/10
720/720 [==============================] - 4s 5ms/step - loss: 0.3377 - accuracy: 0.8654 - val_loss: 0.3768 - val_accuracy: 0.8388
Epoch 

In [24]:
classifier.predict(X_test)

63/63 [==============================] - 0s 884us/step


array([[0.02976845],
       [0.08190811],
       [0.04281131],
       ...,
       [0.03437439],
       [0.05716783],
       [0.56043667]], dtype=float32)

In [28]:
import keras_tuner as kt

In [32]:
#Hyperparameter Tuning
def build_model(hp):
    model = Sequential()
    model.add(Dense(units = hp.Int('units1', min_value=4, max_value=50, step=2),input_dim=11,activation='relu'))
    model.add(Dense(units = hp.Int('units2', min_value=4, max_value=50, step=2),activation='relu'))
    model.add(Dense(units = hp.Int('units3', min_value=4, max_value=50, step=2),activation='relu'))
    model.add(Dense(units=1,activation='sigmoid'))
    model.compile(optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',values=[2e-3,1e-3])), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [41]:
tuner_search = kt.RandomSearch(build_model,objective = 'val_accuracy',max_trials = 10,overwrite=True)
tuner_search.search(X_train,Y_train, epochs=20, validation_split=0.25)

Trial 10 Complete [00h 00m 08s]
val_accuracy: 0.859499990940094

Best val_accuracy So Far: 0.8665000200271606
Total elapsed time: 00h 01m 35s
INFO:tensorflow:Oracle triggered exit


In [42]:
model = tuner_search.get_best_models()

In [43]:
joblib.dump(model,'classifier.pkl')

INFO:tensorflow:Assets written to: ram://7f84a467-4833-4ec6-a537-fc3f0e5d81ef/assets


['classifier.pkl']